<a href="https://colab.research.google.com/github/elshafee/ML_tensorflow/blob/main/Time_Series_Exercise_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import csv
import tensorflow as tf
import numpy as np
import urllib
import math
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass


# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

def model_forecast(model, series, window_size):
   ds = tf.data.Dataset.from_tensor_slices(series)
   ds = ds.window(window_size, shift=1, drop_remainder=True)
   ds = ds.flat_map(lambda w: w.batch(window_size))
   ds = ds.batch(32).prefetch(1)
   forecast = model.predict(ds)
   return forecast


def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
    urllib.request.urlretrieve(url, 'sunspots.csv')

    time_step = []
    sunspots = []

    with open('sunspots.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for row in reader:
            sunspots.append(float(row[2]))
            time_step.append(int(row[0]))

    series = np.array(sunspots)
    # DO NOT CHANGE THIS CODE
    # This is the normalization function
    min = np.min(series)
    max = np.max(series)
    series -= min
    series /= max
    time = np.array(time_step)

    # The data should be split into training and validation sets at time step 3000
    # DO NOT CHANGE THIS CODE
    split_time = 3000


    time_train = time[:split_time]
    x_train = series[:split_time]
    time_valid = time[split_time:]
    x_valid = series[split_time:]

    # DO NOT CHANGE THIS CODE
    window_size = 30
    batch_size = 32
    shuffle_buffer_size = 1000

    train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size,
                                 shuffle_buffer=shuffle_buffer_size)

    print(np.array(train_set).shape)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=60, kernel_size=5,
                               strides=1, padding="causal",
                               activation="relu",
                               input_shape=[None, 1]),
        tf.keras.layers.LSTM(60, return_sequences=True),
        tf.keras.layers.LSTM(60, return_sequences=True),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1)
    ])


    rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
    rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]
    result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()
    test_val = 100 * result
    score = math.ceil(17 - test_val)
    if score > 5:
      score = 5

    print(score)    
    return model

if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")

# THIS CODE IS USED IN THE TESTER FOR FORECASTING. IF YOU WANT TO TEST YOUR MODEL
# BEFORE UPLOADING YOU CAN DO IT WITH THIS






()
Epoch 1/10
93/93 [==============================] - 9s 46ms/step - loss: 0.0793
Epoch 2/10
93/93 [==============================] - 5s 49ms/step - loss: 0.0524
Epoch 3/10
93/93 [==============================] - 5s 49ms/step - loss: 0.0486
Epoch 4/10
93/93 [==============================] - 5s 48ms/step - loss: 0.0464
Epoch 5/10
93/93 [==============================] - 5s 47ms/step - loss: 0.0461
Epoch 6/10
93/93 [==============================] - 4s 46ms/step - loss: 0.0453
Epoch 7/10
93/93 [==============================] - 4s 43ms/step - loss: 0.0451
Epoch 8/10
93/93 [==============================] - 5s 48ms/step - loss: 0.0448
Epoch 9/10
93/93 [==============================] - 5s 48ms/step - loss: 0.0450
Epoch 10/10
93/93 [==============================] - 5s 51ms/step - loss: 0.0448
5


In [ ]:
print(score)

NameError: ignored